# 0. Import thư viện

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

# 1. Data Preprocessing
## 1.0. Data Loading

In [2]:
df = pd.read_csv('data/kill_match_stats.csv')

In [3]:
df.head()

,killed_by,killer_name,killer_placement,killer_position_x,killer_position_y,map,match_id,time,victim_name,victim_placement,victim_position_x,victim_position_y
0,Grenade,KrazyPortuguese,5.0,657725.10,146275.2,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,823,KrazyPortuguese,5.0,657725.10,146275.2
1,SCAR-L,nide2Bxiaojiejie,31.0,93091.37,722236.4,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,194,X3evolution,33.0,92238.68,723375.1
2,S686,Ascholes,43.0,366921.40,421623.9,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,103,CtrlZee,46.0,367304.50,421216.1
3,Down and Out,Weirdo7777,9.0,472014.20,313274.8,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,1018,BlackDpre,13.0,476645.90,316758.4
4,M416,Solayuki1,9.0,473357.80,318340.5,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,1018,Vjolt,13.0,473588.50,318418.8


In [4]:
df.drop(columns = ['killer_name', 'victim_name', 'killer_placement', 'victim_placement', 'match_id'], inplace = True)

In [5]:
df = df[df['map'] == 'ERANGEL'].reset_index(drop = True)

In [6]:
df.drop(columns = ['map'], inplace = True)

In [7]:
df.rename(columns = {
    'killer_position_x': 'kx', 
    'killer_position_y': 'ky',
    'victim_position_x': 'vx',
    'victim_position_y': 'vy'
},inplace = True)

In [8]:
df.head()

,killed_by,kx,ky,time,vx,vy
0,Down and Out,496989.8,312569.7,1035,497385.4,331528.2
1,M16A4,496989.8,312569.7,1035,497819.4,331981.3
2,AKM,460416.7,414748.8,1422,459817.9,414426.3
3,AKM,488034.1,347220.3,1210,487444.2,347651.0
4,SKS,501062.9,425078.6,1818,493043.4,434458.1


<span style="color:red">Câu lệnh bên dưới chỉ thực hiện một lần duy nhất trên máy một thành viên duy nhất. Sau khi thực hiện, tiến hành lưu trữ df thành file kill_match_stats_v1.csv, các thành viên khác chỉ cần load file này vào là được, không cần chạy lại câu lệnh `to_csv` để tránh mất thời gian<span>

In [9]:
df.to_csv('data/kill_match_stats_v1.csv', index = False)

## 1.1. Data Cleaning

In [4]:
df = pd.read_csv('data/kill_match_stats_v1.csv')

In [5]:
df.loc[df['kx'].isnull(), 'kx'] = df['vx']
df.loc[df['ky'].isnull(), 'ky'] = df['vy']

### 1.1.2. Duplicated values

In [6]:
df.duplicated().sum()

257932

Dùng hàm `duplicated` để kiểm tra các dòng trùng nhau, ta thấy có 257932 dòng dữ liệu bị trùng. Tuy nhiên ta có thể quan sát kĩ hơn các dòng này 

In [7]:
data = df[df.duplicated()]

In [8]:
data[['kx', 'ky', 'vx', 'vy']].value_counts()

kx        ky        vx        vy      
0.0       0.0       0.0       0.0         257127
399045.9  300804.1  0.0       0.0              2
482652.2  446492.6  0.0       0.0              2
446353.9  629764.4  446300.3  629816.6         1
444775.8  623105.9  444581.1  623158.9         1
                                           ...  
348630.2  312316.8  350118.1  318378.7         1
349536.5  563234.7  349230.2  564633.3         1
349832.6  565963.8  350066.0  566620.0         1
349866.4  563824.9  354611.3  570966.2         1
738463.6  434072.6  738463.6  434072.6         1
Length: 804, dtype: int64

Quan sát các giá trị khác nhau của các cột tọa độ, ta nhận thấy đa số các cột trùng nhau có tọa độ killer và victim nằm ở (0, 0), nghĩa là ở góc bản đồ, nơi không có địa hình (sẽ được minh họa kỹ hơn ở phần trực quan hóa dữ liệu). Đây có thể là lỗi trong quá trình game vận hành, số lượng các dòng bị lỗi cũng chỉ khoảng 2,5% dữ liệu, ta có thể đơn giản xóa đi các dòng này

In [9]:
df.drop_duplicates(inplace = True, ignore_index = True)

## 1.3 Ourline